In [35]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers.tokenization_utils_base import BatchEncoding

In [11]:
dataset = load_dataset('imdb')

In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [13]:
dataset['train'][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [6]:
MODEL_NAME = 'microsoft/deberta-v3-small'

In [7]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [17]:
MAX_LENGTH = 512

def preprocess_train(sample):
    return tokenizer(sample['text'], truncation=True, max_length=128)


In [18]:
ds_train = dataset['train'].map(preprocess_train).remove_columns(['text'])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [25]:
collator = DataCollatorWithPadding(tokenizer)
def collate_fn(batch):
    return collator(batch)  

train_loader = torch.utils.data.DataLoader(
    ds_train,
    batch_size=4,
    num_workers=0,
    drop_last=True,
    collate_fn=collate_fn,
)

In [27]:
for x in train_loader:
    print(x['input_ids'].size())
    break

torch.Size([4, 128])


In [33]:
device = 'cuda:0'

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME).to(device)

In [40]:
optimizer = torch.optim.AdamW(model.parameters())
optimizer.zero_grad()

In [ ]:
for x in train_loader:
    #model_inputs = BatchEncoding(x).to(device)
    for k in x:
        x[k] = x[k].to(device)
    outputs = model(**x)
    print(outputs.loss.item())
    
    outputs.loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    

In [38]:
outputs

SequenceClassifierOutput(loss=tensor(0.9102, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[-0.0809,  0.3151],
        [-0.0813,  0.3127],
        [-0.0822,  0.3143],
        [-0.0827,  0.3123]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)